In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.layers import Conv2D, MaxPool2D
from keras.models import Model
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import weave

Using TensorFlow backend.


In [2]:
def lr_schedule(epoch):
    return (0.00100 - 7e-6*epoch)

def straight_stack(x, num_filters, rep = 3, weight_decay = 1e-4, dropout_start = 0.2, dropout_add = .1):
    for n in range(rep):
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = MaxPool2D()(x)
        x = Dropout(dropout_start+n*dropout_add)(x)
    return x

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape)
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

batch_size = 64

(50000, 3, 32, 32)


In [4]:
viewer = keras.callbacks.TensorBoard(log_dir='./logs', 
                                     histogram_freq=0, 
                                     batch_size=batch_size, 
                                     write_graph=True)
num_filters = 4
num_val_test = 500
epochs = 30
model_details = {}
conv_layers = 3
for num_stack in [4,2,1]:
    inputs = Input(shape=(3,32,32))
    input_array = []
    pyrm_layers = int(np.log2(num_stack))
    for stack in range(num_stack):
        x = straight_stack(inputs, num_filters/num_stack, rep = conv_layers - pyrm_layers)
        input_array.append(x)
    if pyrm_layers > 0:
        input_size = tuple(input_array[0].shape[1:].as_list())
        print(input_size)
        x = weave.pyrm_net(input_size=input_size,
                           n_layers = pyrm_layers,
                           n_filters_start = input_size[0]*2,
                           n_gpus = 1,
                           inputs = input_array,
                           r_filter = 2,
                           r_combine= 1,
                           min_dim = 2,
                           max_pool_loc = 2,
                           pure_combine = True,
                           end_max_pool = False)
    else:
        pass
    print("output shape of conv layers: ",x.shape)
    x = Flatten()(x)
    
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=[inputs], outputs=predictions)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.summary()
    num_params = model.count_params()
    print('number of parameters %d', num_params)
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test[0:num_val_test],y_test[0:num_val_test]),
                    callbacks=[LearningRateScheduler(lr_schedule), viewer])
    model_details[num_stack] = (num_params, model.evaluate(x_test,y_test))
    print(model_details)
    
    
    

(1, 16, 16)
GPU settings allow for inf layers
Minimum output size allow for 6 layers
Number of layers 2
First Layer Size: 2 (Number of Units)
('Disjoint:', True)
number of units 2
number of devices 4
number of inputs 4
First Layer Output Size: 2 (Number of Tensors)
('Disjoint:', True)
number of units 1
number of devices 4
number of inputs 2
Final Layer Size 1
('output shape of conv layers: ', TensorShape([Dimension(None), Dimension(4), Dimension(16), Dimension(16)]))
('number of parameters %d', 11926)
Epoch 1/30
781/781 [==============================] - 516s 660ms/step - loss: 1.9272 - acc: 0.2974 - val_loss: 1.7496 - val_acc: 0.3720
Epoch 2/30
781/781 [==============================] - 527s 675ms/step - loss: 1.7273 - acc: 0.3730 - val_loss: 1.6566 - val_acc: 0.3820
Epoch 3/30
781/781 [==============================] - 527s 674ms/step - loss: 1.6687 - acc: 0.3967 - val_loss: 1.5621 - val_acc: 0.4480
Epoch 4/30
781/781 [==============================] - 523s 670ms/step - loss: 1.6291 

781/781 [==============================] - 436s 558ms/step - loss: 1.3336 - acc: 0.5205 - val_loss: 1.5249 - val_acc: 0.4460
Epoch 24/30
781/781 [==============================] - 435s 557ms/step - loss: 1.3338 - acc: 0.5221 - val_loss: 1.4937 - val_acc: 0.4700
Epoch 25/30
781/781 [==============================] - 436s 558ms/step - loss: 1.3295 - acc: 0.5239 - val_loss: 1.3993 - val_acc: 0.4860
Epoch 26/30
781/781 [==============================] - 435s 557ms/step - loss: 1.3289 - acc: 0.5253 - val_loss: 1.4573 - val_acc: 0.4900
Epoch 27/30
781/781 [==============================] - 436s 558ms/step - loss: 1.3270 - acc: 0.5263 - val_loss: 1.4558 - val_acc: 0.4840
Epoch 28/30
781/781 [==============================] - 435s 557ms/step - loss: 1.3186 - acc: 0.5309 - val_loss: 1.4503 - val_acc: 0.4760
Epoch 29/30
781/781 [==============================] - 436s 558ms/step - loss: 1.3175 - acc: 0.5280 - val_loss: 1.4663 - val_acc: 0.4760
Epoch 30/30
10000/10000 [============================

In [5]:
model_details


{1: (7646, [1.2174990461349486, 0.5706]),
 2: (6894, [1.4662439916610719, 0.5037]),
 4: (11926, [1.3469636142730712, 0.5209])}

In [9]:
num_filters = 32
num_val_test = 500
epochs = 65
model_details = {}
conv_layers = 3
num_stack = 4
inputs = Input(shape=(3,32,32))
input_array = []
pyrm_layers = 2
for stack in range(num_stack):
    x = straight_stack(inputs, num_filters/num_stack, rep = 1)
    input_array.append(x)
input_size = tuple(input_array[0].shape[1:].as_list())
print(input_size)
x = weave.pyrm_net(input_size=input_size,
                   n_layers = pyrm_layers,
                   n_filters_start = input_size[0]*2,
                   n_gpus = 1,
                   inputs = input_array,
                   r_filter = 2,
                   r_combine= 1,
                   min_dim = 2,
                   max_pool_loc = 1,
                   pure_combine = True,
                   end_max_pool = False)

print("output shape of conv layers: ",x.shape)
x = Flatten()(x)

predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=[inputs], outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


(8, 16, 16)
GPU settings allow for inf layers
Minimum output size allow for 3 layers
Number of layers 2
First Layer Size: 2 (Number of Units)
('Disjoint:', True)
number of units 2
number of devices 4
number of inputs 4
First Layer Output Size: 2 (Number of Tensors)
('Disjoint:', True)
number of units 1
number of devices 4
number of inputs 2
Final Layer Size 1
('output shape of conv layers: ', TensorShape([Dimension(None), Dimension(32), Dimension(8), Dimension(8)]))
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 3, 32, 32)    0                                            
__________________________________________________________________________________________________
conv2d_55 (Conv2D)              (None, 8, 32, 32)    224         input_6[0][0]                    
__________________________________

In [ ]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                steps_per_epoch=x_train.shape[0] // batch_size,
                epochs=epochs,
                verbose=1,
                validation_data=(x_test[0:num_val_test],y_test[0:num_val_test]),
                callbacks=[LearningRateScheduler(lr_schedule)])
model.evaluate(x_test,y_test)

Epoch 1/65
 66/781 [=>............................] - ETA: 43:11 - loss: 2.0340 - acc: 0.2628